First part of script consists of raw data processing. Scroll down till text in **bold**

In [311]:
import numpy as np
import pandas as pd
import glob, os
from scipy.stats import norm 

In [312]:
pd.set_option('display.width', 150)

In [313]:
path = r'C:\Users\БФедин\OneDrive - GENERIX\InventoryReports\Baltika'
all_files = glob.glob(path + "/*.txt")

In [314]:
col_specification = [(0,2),(2,15),(15,28),(28,30),
                     (30,34),(34,36),(36,38),(38,42),(42,52),(52,62)]

In [315]:
li = []

for filename in all_files:
    df = pd.read_fwf(filename, header=None,colspecs=col_specification,float_format='%.f')
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)

df = df.dropna() # Script drops N/A values. They may appear if file has blank lines.

df.columns = ['Movement_Code','GLN_Code','EAN_Code','Site_Item_Type',
                  'Case_Count','Movement_Day','Movement_Month','Movement_Year',
                    'Quantity','Base_Unit_Code']

In [316]:
df[['GLN_Code','EAN_Code','Base_Unit_Code','Case_Count']] = \
df[['GLN_Code','EAN_Code','Base_Unit_Code','Case_Count']].astype(str).replace('\.0', '', regex=True)

df['Quantity'] = df['Quantity'].astype(int)

In [317]:
df['Movement_Day'] = df['Movement_Day'].astype(str).replace('\.0', '', regex=True)
df['Movement_Day'] = df['Movement_Day'].str.zfill(2)
df['Movement_Month'] = df['Movement_Month'].astype(str).replace('\.0', '', regex=True)
df['Movement_Month'] = df['Movement_Month'].str.zfill(2)
df['Movement_Year'] = df['Movement_Year'].apply(str).replace('\.0', '', regex=True)
df['Movement_Date'] = df["Movement_Day"] + "-" + df["Movement_Month"] + "-" + df["Movement_Year"]
df['Movement_Date'] = df['Movement_Date'].apply(str).replace('\.0', '', regex=True)
df['Movement_Date'] = pd.to_datetime(df.Movement_Date,format = '%d-%m-%Y', dayfirst=True)
df = df.drop(['Movement_Day','Movement_Month','Movement_Year'], axis=1) 

In [318]:
if 5 in df['Movement_Code']:
    df['Movement_Code'] = df['Movement_Code'].replace([1,4,3,5],
                                                      ['Stock','Transit','Shipped','OOS'])
else: 
    df['Movement_Code'] = df['Movement_Code'].replace([1,4,3],
                                                      ['Stock','Transit','Shipped'])

In [319]:
df['Site_Item_Type'] = df['Site_Item_Type'].replace([0,1],
                                                    ['Standard','Promo'])

In [320]:
df = df[['Movement_Code','GLN_Code','EAN_Code','Site_Item_Type',
                  'Case_Count','Movement_Date','Quantity','Base_Unit_Code']]

In [321]:
dfPivot = pd.pivot_table(df,aggfunc='sum',values='Quantity',
                         index=['Movement_Date','EAN_Code','Base_Unit_Code'],
                         columns='Movement_Code').reset_index()

In [322]:
dfPivot = dfPivot.fillna(0)

In [323]:
Movement_Codes_Order = ['Movement_Week','Movement_Date','EAN_Code','Base_Unit_Code',
                        'Stock','Transit','Shipped','OOS','Demand']

dfPivot = dfPivot.reindex(Movement_Codes_Order,axis=1)

In [324]:
dfPivot['Movement_Week'] = 'cw'+dfPivot['Movement_Date'].dt.strftime('%V')

In [325]:
if 'OOS' in df['Movement_Code'].unique():
    dfPivot['OOS'] = dfPivot['OOS'].fillna(0)
    dfPivot['Demand'] = dfPivot['Shipped'] + dfPivot['OOS']
else:
    dfPivot['OOS'] = dfPivot['OOS'].fillna(0)
    dfPivot['Demand'] = dfPivot['Shipped']

In [326]:
dfPivot = dfPivot.rename_axis("Line_ID",axis=1)

In [327]:
dfPivot.iloc[:, 4:] = dfPivot.iloc[:, 4:].astype(int)

In [328]:
dfPivot = dfPivot.sort_values(by=['Movement_Date'],ascending=False,inplace=False)

In [329]:
dfPivot.reset_index(drop=True,inplace=True)
dfPivot.sort_index(ascending=False,inplace=True)

--- **Safety Stock Calculation script starts here** ---

Product chosen for this script is one of supplier's topsellers. It's is rarely included in promo activities.

In [330]:
dfPivot = dfPivot[dfPivot['Base_Unit_Code']=='1000253382']

We choose calendar week as a period. 

In [331]:
dfOneProduct = dfPivot.groupby('Movement_Week',as_index=False)[['Stock','Shipped','OOS','Demand']].sum()

Aggregated quantities can be observed in the following table.

In [332]:
print(dfOneProduct[['Movement_Week','Demand']].tail(10))

Line_ID Movement_Week  Demand
11               cw44   41607
12               cw45   37393
13               cw46   62100
14               cw47   60111
15               cw48   94296
16               cw49   75029
17               cw50   69098
18               cw51    9144
19               cw52   27278
20               cw53    9279


*Z* is a demand threshold. Below we will calculate the probability for an occurence of demand quantity to be below *Z* value within a chosen period.

In [333]:
z = 50000 # Random value chosen as a threshold. Just a bit above of the mean value.

*MU* is a mean value for product's demand. In our case it's calculated based on data since August 16th.

In [334]:
mu = np.round(dfOneProduct['Demand'].mean(),decimals=2) # mean value

*Sigma* is standard deviation value for product's demand.

In [335]:
sigma = np.round((np.std(dfOneProduct['Demand'],ddof=1)),decimals=2) # standard deviation

In [336]:
print('\n','z:',z,'\n','mu:',mu,'\n','sigma:',sigma)


 z: 50000 
 mu: 47933.9 
 sigma: 21003.95


Below we compute the CDF of a normal distribution - *Alpha*. It's calculated based on inputs above only.

Later we will create a table with an extended row of CDF levels.

In [337]:
alpha = np.round(norm.cdf(z,mu,sigma),decimals=2)

In [338]:
print('alpha: ',alpha)

alpha:  0.54


**Calculation of the inverse CDF of a normal distribution.** 

It will give us the threshold *z* that will achieve a probability *α* for an occurrence of the distribution *𝒳* to be below this threshold *z*. 

Here the use the same *alpha* value as in the example above.

In [339]:
z1 = norm.ppf(alpha,mu,sigma)

In [340]:
print('Inverse CDF: ', np.round(z1,decimals=0))

Inverse CDF:  50043.0


Now we can calculate the relationship between several service levels and required stock levels. Here we use variable *alpha1*. It represents an array of CDF values.

In [341]:
alpha1 = np.array([0.5,0.6,0.7,0.8,0.9,0.95,0.965,0.975,0.985,0.99])
inv = norm.ppf(alpha1,mu,sigma)

In [342]:
table1 = {'Inventory level, pcs': inv.astype(int),
         'Cycle service level, %': alpha1*100}

df1 = pd.DataFrame(table1)
df1.set_index('Cycle service level, %', inplace=True)
dfT1 = df1.transpose()

*Table 1* below shows the relationship between the inventory level at the beginning of a period (1 week) and the expected cycle service level.

In [343]:
print(dfT1)

Cycle service level, %   50.0   60.0   70.0   80.0   90.0   95.0   96.5   97.5   98.5   99.0
Inventory level, pcs    47933  53255  58948  65611  74851  82482  85991  89100  93514  96796


**Calculation of a Service level factor.**

Service level factor is the ratio that multiplies the demand deviation in order to compute the required safety stock and to obtain a desired service level.

In [344]:
table2 = {'Service level factor': np.round(norm.ppf(alpha1),decimals=2),
          'Cycle service level, %': alpha1*100}

df2 = pd.DataFrame(table2)
df2.set_index('Cycle service level, %', inplace=True)
dfT2 = df2.transpose()

*Table 2* below shows the relationship between the desired cycle service level and its service level factor. Impact of service level factor becomes bigger as you approach 100% service level. A higher cycle service level implies bigger extra costs.

In [345]:
print(dfT2)

Cycle service level, %  50.0  60.0  70.0  80.0  90.0  95.0  96.5  97.5  98.5  99.0
Service level factor     0.0  0.25  0.52  0.84  1.28  1.64  1.81  1.96  2.17  2.33


**Cycle and Safety stocks' calculation.**

Value for cycle stock is taken from Table 1 and corresponds to 95% cycle service level.

In [346]:
InventoryLevel = dfT1.loc[['Inventory level, pcs'],95].values[0]

If we want to calculate a safety stock in order to perform with a 95% service level, then we should multiply Service level factor (*1.64*) by Demand deviation (*sigma*).

In [347]:
# SafetyStock = (1.64*sigma).astype(int) # This code also works, but it implies 
                                         # a manual search for necessary service level factor.

SLFactor = dfT2.loc[['Service level factor'],95].values[0]  # 95 can be replaced with a desired service level.

SafetyStock = (SLFactor*sigma).astype(int)

Cycle Stock equals to Inventory level minus Safety Stock. 

In [348]:
CycleStock = InventoryLevel - SafetyStock

All three types of stock are displayed below.

In [349]:
table3 = {'Safety stock, pcs': SafetyStock,
          'Cycle stock, pcs': CycleStock,
          'Inventory level, pcs': InventoryLevel}

df3 = pd.DataFrame(table3,index=[''])
dfT3 = df3.transpose()
print(dfT3)

                           
Safety stock, pcs     34446
Cycle stock, pcs      48036
Inventory level, pcs  82482


**Summary:**

Inventory level calculated above is likely to be sufficient for performing on a 95% service level.

According to inputs, we calculate replenishment based on weekly demand. In this particular case, we don't take Lead time into account.

A more complex approach to stock calculation will be presented in the next document.